In [15]:
options(repr.plot.width=14, repr.plot.height=12,unit="cm")

# Raw data extraction and visualisation

The following code, extracts, loads, plots, and subsets the data that will be used in the analysis and to prepare visualisations. This includes:

 -  data on global production of buckwhat, available from EarthStat http://www.earthstat.org/
 -  outline of the continents, available from Narural Eart (https://www.naturalearthdata.com/downloads/50m-physical-vectors/)
 -  administrative division of China (http://biogeo.ucdavis.edu/data/diva/adm/CHN_adm.zip)
 -  data on climatic conditions for the past 120 000 years available from "https://ndownloader.figshare.com/files/22659026"
 -  dataset with the locations of fagopyrum macro and micro remains from China, collated from ..., and available in the project's repository.

In [2]:
### Load libraries:
library(here) # setting paths
library(raster) # loading spatial data
library(rgdal) # loading spatial data
library(tmap) # thematic mapping
library(ncdf4) # loading climate data
library(stringr) # wraps labels on the maps

here() starts at G:/My Drive/SDM_China

Loading required package: sp

rgdal: version: 1.5-18, (SVN revision 1082)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.0.4, released 2020/01/28
Path to GDAL shared files: D:/Programs/R-4.0.3/library/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 6.3.1, February 10th, 2020, [PJ_VERSION: 631]
Path to PROJ shared files: D:/Programs/R-4.0.3/library/rgdal/proj
Linking to sp version:1.4-4
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.



In [3]:
### Load utility functions that will be used in data processing:
source(here("R","extractRastersFromNetCDF.r"))

In [66]:
### Define urls from which to download data:

# Production data
prod_url <- "https://s3.us-east-2.amazonaws.com/earthstatdata/HarvestedAreaYield175Crops_Indvidual_Geotiff/buckwheat_HarvAreaYield_Geotiff.zip"
# Also define the name of the folder with the production data, taken from the path
prod_fname<-here("raw_data","/buckwheat_HarvAreaYield_Geotiff")

# Outline of the continents
cont_url<-"https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/physical/ne_50m_land.zip"
# Also define the name of the folder with continents, taken from the path
cont_fname<-here("raw_data","ne_50m_land")

# Administrative division of China
china_url<-"http://biogeo.ucdavis.edu/data/diva/adm/CHN_adm.zip"
# Define the directory to which the files will be extracted
china_fname <- here("raw_data","CHN_adm")

# Environmental variables
env_url <- "https://ndownloader.figshare.com/files/22659026"
# Define the directory to which the files will be extracted
env_fname <- here("raw_data","22659026")

In [64]:
### Define the output paths:

# path to raw data folder
path2raw_data<- here("raw_data")

# paths to production data
path2clipped_china<-here("data","buckwheat_production","clipped")
path2masked_china<-here("data","buckwheat_production","masked")

# paths to production plots
path2world_production <- here("outputs","01_01_World_buckwheat_production.tiff")
path2china_production <- here("outputs","01_02_China_buckwheat_production.tiff")

# paths to environmental variables table:

path2env_table<-here("outputs","01_03_Predictor_variables.csv")

# paths to past environmental layers:

path2past_env<-here("data","environmental","past")
path2present_env<-here("data","environmental","present")

## Present distribution of buckwheat:

### Global distribution

Data on the present global production of buckwhat is available from EarthStat (http://www.earthstat.org/) and contains information on the volume of production, cultivated area and yield, as well as data quality. In the model it will be used as a proxy for the suitabiliy of environment for buckwheat cultivation and will be used as the response variable. The following code downloads the data from EarthStat (https://www.naturalearthdata.com/downloads/50m-physical-vectors/) and extracts the individual files from the archive. Then it also downloads and loads the files with the outline of the continents, to serve as a background for data visualisation.

In [ ]:
### Production data
## Download and exract data:
# Download the zipped folder with the production data
download.file(prod_url,paste(path2raw_data,prod_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
prod_files <- unzip(paste(path2raw_data,prod_fname,".zip",sep=""),exdir=path2raw_data)
# Removes the zipped folder
file.remove(paste(path2raw_data,prod_fname,".zip",sep=""))

In [6]:
## Load data into workspace:
# List all layers related to buckwheat production
prod_layers<-list.files(path=prod_fname,pattern='tif$',full.names=TRUE)
# Put all the layers into a raster stack
prd<-stack(prod_layers)

In [419]:
### Download the shapefile of the continents as well:
download.file(url,paste(cont_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
dir.create(cont_fname)
files <- unzip(paste(cont_fname,".zip",sep=""),exdir=cont_fname)
# Removes the zipped folder:#
file.remove(paste(cont_fname,".zip",sep=""))

Warning message in dir.create(cont_fname):
"'G:\My Drive\SDM_China\raw_data\ne_50m_land' already exists"


[1] TRUE

In [7]:
### Load the shapefile with the outline of the continents
continents <- readOGR(dsn = cont_fname, layer = sub('.*/', '', cont_fname))

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\SDM_China\raw_data\ne_50m_land", layer: "ne_50m_land"
with 1420 features
It has 3 fields
Integer64 fields read as strings:  scalerank 


In [23]:
## Get the names for plots:
# Look at the existing names of the layers in the raster stack
names(prd)

[1] "buckwheat_DataQuality_HarvestedArea" "buckwheat_DataQuality_Yield"        
[3] "buckwheat_HarvestedAreaFraction"     "buckwheat_HarvestedAreaHectares"    
[5] "buckwheat_Production"                "buckwheat_YieldPerHectare"

In [8]:
# Create a list of more informative names
prod_names <- c("Data quality: harvested area","Data quality: yield","Harvested area: fractional","Harvested area: hectares",
            "Production in tons","Yield (tons per hectare)")

In [9]:
census_level <-c("missing census data","country level census data","interpolated from within 2° lat/lon","state level census data","county level census data")

In [10]:
#Plot data on bucwheat production
#This prepares data for visualisation
## Define breakpoints for each of the map scales
#breakpoints <- lapply(as.list(prd[[c(1,2,3,5,4,6)]]),function(x){return(classIntervals(x[!is.na(x)], n = 50, style = "quantile"))})
#Transform the shapefile with continents for visualisation
#continents <- spTransform(continents, crs(prd))
#Make a color palette for visualisation
pal <- colorRampPalette(c("lightyellow","orange","orange3","saddlebrown"))
# Set NA's to the cells that represent the seas
prd[[1]][is.na(prd[[3]])]<-NA
prd[[2]][is.na(prd[[3]])]<-NA

In [58]:
## Plot global production data

# Make panels for the data quality:

gprod1<-tm_shape(prd[[c(1,2)]]) + # Order the maps in a sensible way
    tm_grid(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.6)+
    #tm_compass()+
    # Add raster layer:
    tm_raster(labels=str_wrap(census_level,40),style=c("cat","cat"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=1,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    tm_layout(panel.labels=prod_names[1:2],
              panel.label.bg.color="white",
              frame=FALSE,
              frame.lwd = NA,
              legend.position=c(0,0),
              attr.outside=TRUE,
              panel.label.fontface="bold",
              panel.label.height=2,
              panel.label.size=0.7,
              legend.text.size=0.45
             )

# Make panels for the production data:

gprod2<-tm_shape(prd[[c(3,5,4,6)]]) + # Order the maps in a sensible way
    tm_grid(lwd = 0.05,col="grey75",n.x=2,n.y=2,labels.size = 0.8)+
    #tm_compass()+
    # Add raster layer:
    tm_raster(style=c("cont","cont","cont","cont"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=2,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(continents)+
    tm_borders(col="grey85")+
    tm_layout(panel.labels=prod_names[c(3,5,4,6)],
              panel.label.bg.color="white",
              frame=FALSE,
              frame.lwd = NA,
              legend.position=c(0,0),
              attr.outside=TRUE,
              panel.label.fontface="bold",
              panel.label.height=2,
              panel.label.size=1,
              legend.text.size=0.6
             )
# Arrenge the maps together
gprod<-tmap_arrange(gprod1, gprod2, heights=c(1/3,2/3))

In [59]:
width=21
height=18
tmap_save(tm = gprod, filename = path2world_production, width = width, height = height,units = "cm",dpi = 300)

stars object downsampled to 1414 by 707 cells. See tm_shape manual (argument raster.downsample)

stars object downsampled to 1414 by 707 cells. See tm_shape manual (argument raster.downsample)

stars object downsampled to 1414 by 707 cells. See tm_shape manual (argument raster.downsample)

stars object downsampled to 1414 by 707 cells. See tm_shape manual (argument raster.downsample)

Map saved to G:\My Drive\SDM_China\outputs\01_01_World_buckwheat_production.tiff

Resolution: 2480.315 by 2125.984 pixels

Size: 8.267717 by 7.086614 inches (300 dpi)



### Distribution in China

This sections downloads the shapefile with the administrative division of China from http://biogeo.ucdavis.edu/data/diva/adm/CHN_adm.zip, which is then used to crop, mask and plot data on buckwheat production in China. The cropped and masked subests of data are saved as GeoTiffs.

In [ ]:
# Download the zipped folder
download.file(china_url,paste(china_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
dir.create(china_fname)
files <- unzip(paste(china_fname,".zip",sep=""),exdir=china_fname)
# Removes the zipped folder:#
file.remove(paste(china_fname,".zip",sep=""))

In [28]:
### Load the shapefile with the borders of China
china <- readOGR(dsn = china_fname, layer = "CHN_adm0")

OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\SDM_China\raw_data\CHN_adm", layer: "CHN_adm0"
with 1 features
It has 70 fields
Integer64 fields read as strings:  ID_0 OBJECTID_1 


In [67]:
### Production data:
# Create directories for subset data
dir.create(path2clipped_china)
dir.create(path2masked_china)

Warning message in dir.create(path2clipped_china):
"'G:\My Drive\SDM_China\data\buckwheat_production\clipped' already exists"
Warning message in dir.create(path2masked_china):
"'G:\My Drive\SDM_China\data\buckwheat_production\masked' already exists"


In [ ]:
# Note that china has the same projection as the prd maps, so it does not need to be reprojected
# Clip the data to the extent of china
clipped_prd<-crop(prd, extent(china), snap="out")
# Save clipped data as GeoTiffs:
writeRaster(clipped_prd,filename=paste(path2clipped_china,"//",names(prd),".tif",sep=""),format="GTiff", overwrite=TRUE,bylayer=TRUE)
# Mask the data to the borders of China
masked_prd<-mask(clipped_prd, china)
# Saved masked data as GeoTiffs
writeRaster(masked_prd,filename=paste(path2masked_china,"//",names(prd),".tif",sep=""),format="GTiff", overwrite=TRUE,bylayer=TRUE)

In [62]:
## Plot production just for China

# Make panels for the data quality:

cprod1<-tm_shape(masked_prd[[c(1,2)]]) + # Order the maps in a sensible way
    tm_grid(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.55)+
    #tm_compass()+
    # Add raster layer:
    tm_raster(labels=str_wrap(census_level,40),style=c("cat","cat"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=1,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(china)+
    tm_borders(col="grey85")+
    tm_layout(panel.labels=prod_names[1:2],
              panel.label.bg.color="white",
              frame=FALSE,
              frame.lwd = NA,
              legend.position=c(0,0),
              attr.outside=TRUE,
              panel.label.fontface="bold",
              panel.label.height=2,
              panel.label.size=0.7,
              legend.text.size=0.45
             )

# Make panels for the production data:

cprod2<-tm_shape(masked_prd[[c(3,5,4,6)]]) + # Order the maps in a sensible way
    tm_grid(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    #tm_compass()+
    # Add raster layer:
    tm_raster(style=c("cont","cont","cont","cont"),palette=pal(4),legend.reverse = TRUE,title="",interval.closure="left",showNA=FALSE,colorNA="NA",midpoint=NULL)+
    # Make facets
    tm_facets(free.scales=TRUE,free.coords=TRUE,ncol=2,nrow=2,free.scales.raster=TRUE)+
    # Add continents outline
    tm_shape(china)+
    tm_borders(col="grey85")+
    tm_layout(panel.labels=prod_names[c(3,5,4,6)],
              panel.label.bg.color="white",
              frame=FALSE,
              frame.lwd = NA,
              legend.position=c(0,0),
              attr.outside=TRUE,
              panel.label.fontface="bold",
              panel.label.height=2,
              panel.label.size=1,
              legend.text.size=0.6
             )
# Arrenge the maps together
cprod<-tmap_arrange(cprod1, cprod2, heights=c(1/3,2/3))

In [63]:
width=21
height=18
tmap_save(tm = cprod, filename = path2china_production, width = width, height = height,units = "cm",dpi = 300)

Map saved to G:\My Drive\SDM_China\outputs\01_02_China_buckwheat_production.tiff

Resolution: 2480.315 by 2125.984 pixels

Size: 8.267717 by 7.086614 inches (300 dpi)



## Climate data data for the past 6000K

Data on the climatic conditions for the past 120 000 years is available from https://figshare.com/s/f098cc85074722ec4930). It has been published alongside the paper () and will be used to define predictor variables in the model. The following code downloads the data, extracts them from the archive, extract relavant layers from the NetCDF file and saves them as tiff files for easier access.

In [69]:
### Preparing environmental data for 6 time slices from Mid-Holocene to the present:
# Download the zipped folder
#download.file(env_url,paste(env_fname,".zip",sep=""), mode = "wb")
# Unzip the folder and get the names of its contents:
files <- unzip(paste(env_fname,".zip",sep=""), exdir=env_fname)
# Removes the zipped folder:
file.remove(paste(env_fname,".zip",sep=""))

Warning message in unzip(paste(env_fname, ".zip", sep = ""), exdir = env_fname):
"error 1 in extracting from zip file"


[1] TRUE

In [71]:
env_fname

[1] "G:/My Drive/SDM_China/raw_data/22659026"

In [70]:
paste(env_fname,".zip",sep="")

[1] "G:/My Drive/SDM_China/raw_data/22659026.zip"

In [4]:
##Extracts the relevant layers from NetCDF and saves them as raster:
# Open the NetCDF, in order to get the names of the variables that are stored there and summarizes the time dimension of the data
env_nc <- ncdf4::nc_open(files[1])
# This extracts the availabe variable names:
var_names <- names(env_nc$var)

[1] "years since 1950-1-1 00:00:00"

 int [1:72(1d)] -120000 -118000 -116000 -114000 -112000 -110000 -108000 -106000 -104000 -102000 ...


In [20]:
### Prepare the table with the shortcuts of environmental variables and their full names:
short_name <-var_names[c(9,11:27)]
full_name <-c("Net Primary Production",'BIO1 Annual Mean Temperature',
                'BIO4 Temperature Seasonality', 
                'BIO5 Max Temperature of Warmest Month',
                'BIO6 Min Temperature of Coldest Month', 
               'BIO7 Temperature Annual Range', 
               'BIO8 Mean Temperature of Wettest Quarter',
               'BIO9 Mean Temperature of Driest Quarter',
               'BIO10 Mean Temperature of Warmest Quarter', 
               'BIO11 Mean Temperature of Coldest Quarter', 
               'BIO12 Annual Precipitation', 
               'BIO13 Precipitation of Wettest Month', 
              'BIO14 Precipitation of Driest Month',
              'BIO15 Precipitation Seasonality',
               'BIO16 Precipitation of Wettest Quarter',
               'BIO17 Precipitation of Driest Quarter', 
               'BIO18 Precipitation of Warmest Quarter',
               'BIO19 Precipitation of Coldest Quarter')
env_table <- cbind(short_name, full_name)
write.csv(env_table, file=path2env_table,row.names=FALSE)

In [ ]:
## Get the times units
env_nc$dim$time$units
str(env_nc$dim$time$vals)

In [44]:
# Define the past time slices for the past 8000 years + 15000 BP
timeslices <-c(seq(1000,8000,by=1000),15000)
# Define the area to mask and crop the rasters by
area<-china

In [25]:
#Extracts the relevant layers from the NetCDF file, using the utility function provided in this repository
lapply(var_names,extractRastersFromNetCDF,timeslice=timeslices,area=area,path2layers=path2past_env,
       path2present=path2present_env,crop=TRUE,mask=TRUE )

Warning message in .rasterObjectFromCDF(x, type = objecttype, band = band, ...):
""level" set to 1 (there are 12 levels)"
Warning message in dir.create(path2layers):
"'data\environmental\past\temperature' already exists"
Warning message in dir.create(paste(path2layers, "cropped//", sep = "")):
"'data\environmental\past\temperature\cropped' already exists"
Warning message in dir.create(paste(path2layers, "masked//", sep = "")):
"'data\environmental\past\temperature\masked' already exists"
Warning message in dir.create(paste(path2present, "cropped//", sep = "")):
"'data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "masked//", sep = "")):
"'data\environmental\present\masked' already exists"
Warning message in .rasterObjectFromCDF(x, type = objecttype, band = band, ...):
""level" set to 1 (there are 12 levels)"
Warning message in dir.create(path2layers):
"'data\environmental\past\precipitation' already exists"
Warning message in dir.creat

Warning message in dir.create(paste(path2layers, "masked//", sep = "")):
"'data\environmental\past\BIO5\masked' already exists"
Warning message in dir.create(paste(path2present, "cropped//", sep = "")):
"'data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "masked//", sep = "")):
"'data\environmental\present\masked' already exists"
Warning message in dir.create(path2layers):
"'data\environmental\past\BIO6' already exists"
Warning message in dir.create(paste(path2layers, "cropped//", sep = "")):
"'data\environmental\past\BIO6\cropped' already exists"
Warning message in dir.create(paste(path2layers, "masked//", sep = "")):
"'data\environmental\past\BIO6\masked' already exists"
Warning message in dir.create(paste(path2present, "cropped//", sep = "")):
"'data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "masked//", sep = "")):
"'data\environmental\present\masked' already exists"
Warning me

Warning message in dir.create(path2layers):
"'data\environmental\past\BIO19' already exists"
Warning message in dir.create(paste(path2layers, "cropped//", sep = "")):
"'data\environmental\past\BIO19\cropped' already exists"
Warning message in dir.create(paste(path2layers, "masked//", sep = "")):
"'data\environmental\past\BIO19\masked' already exists"
Warning message in dir.create(paste(path2present, "cropped//", sep = "")):
"'data\environmental\present\cropped' already exists"
Warning message in dir.create(paste(path2present, "masked//", sep = "")):
"'data\environmental\present\masked' already exists"


[[1]]
[1] "data\\environmental\\past\\temperature\\"
[2] "data\\environmental\\present\\"          

[[2]]
[1] "data\\environmental\\past\\precipitation\\"
[2] "data\\environmental\\present\\"            

[[3]]
[1] "data\\environmental\\past\\cloud\\" "data\\environmental\\present\\"    

[[4]]
[1] "data\\environmental\\past\\relative_humidity\\"
[2] "data\\environmental\\present\\"                

[[5]]
[1] "data\\environmental\\past\\wind_speed\\"
[2] "data\\environmental\\present\\"         

[[6]]
[1] "data\\environmental\\past\\min_temperature\\"
[2] "data\\environmental\\present\\"              

[[7]]
[1] "data\\environmental\\past\\max_temperature\\"
[2] "data\\environmental\\present\\"              

[[8]]
[1] "data\\environmental\\past\\biome\\" "data\\environmental\\present\\"    

[[9]]
[1] "data\\environmental\\past\\npp\\" "data\\environmental\\present\\"  

[[10]]
[1] "data\\environmental\\past\\lai\\" "data\\environmental\\present\\"  

[[11]]
[1] "data\\environmental\\past\\BIO1\\" "data\\environmental\\present\\"   

[[12]]
[1] "data\\environmental\\past\\BIO4\\" "data\\environmental\\present\\"   

[[13]]
[1] "data\\environmental\\past\\BIO5\\" "data\\environmental\\present\\"   

[[14]]
[1] "data\\environmental\\past\\BIO6\\" "data\\environmental\\present\\"   

[[15]]
[1] "data\\environmental\\past\\BIO7\\" "data\\environmental\\present\\"   

[[16]]
[1] "data\\environmental\\past\\BIO8\\" "data\\environmental\\present\\"   

[[17]]
[1] "data\\environmental\\past\\BIO9\\" "data\\environmental\\present\\"   

[[18]]
[1] "data\\environmental\\past\\BIO10\\" "data\\environmental\\present\\"    

[[19]]
[1] "data\\environmental\\past\\BIO11\\" "data\\environmental\\present\\"    

[[20]]
[1] "data\\environmental\\past\\BIO12\\" "data\\environmental\\present\\"    

[[21]]
[1] "data\\environmental\\past\\BIO13\\" "data\\environmental\\present\\"    

[[22]]
[1] "data\\environmental\\past\\BIO14\\" "data\\environmental\\present\\"    

[[23]]
[1] "data\\environmental\\past\\BIO15\\" "data\\environmental\\present\\"    

[[24]]
[1] "data\\environmental\\past\\BIO16\\" "data\\environmental\\present\\"    

[[25]]
[1] "data\\environmental\\past\\BIO17\\" "data\\environmental\\present\\"    

[[26]]
[1] "data\\environmental\\past\\BIO18\\" "data\\environmental\\present\\"    

[[27]]
[1] "data\\environmental\\past\\BIO19\\" "data\\environmental\\present\\"

## Plot locations of past fagopyrum macro and microremains

The dataset with the locations of fagopyrum macro and micro remains from the past, was complied by ... . The original dataset was supplemented with additional information published since then in a csv file ' '  provided in this repository. The code below plots the data to create figure ... in the paper.

In [ ]:
### Fagopyrum locations for China:
#Import the table for Fagopyrum occurance records, based on (Hunt et al., 2017) and further evidence from the literature
loc_ea <- read.csv("raw_data//fagopyrum_east_asia.csv")
# Trnsforma data into a spatial points data.frame
coordinates(loc_ea)<-~longitude+latitude

In [ ]:
# Plot locations of macro and macrofossils 